In [1]:
import pdf2image
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
from pytesseract import Output

import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import csv

%matplotlib auto
import re
import csv

Using matplotlib backend: <object object at 0x15af08b90>


In [2]:
# now, highlight keyword area found on the image
def find_area(corr_ls,file_):
    global image
    global cache
    image = cv2.imread(file_)
    
    cache = image.copy()
    # ignore first crop as it refers to the whole page
    invoice = cv2.imread(file_)
    
    for i,corr in enumerate(corr_ls):
        image = cache.copy()
        x,y,w,h = corr
        cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.imshow('crop', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows() # destroys the window showing image
        cv2.waitKey(1) #for mac OS

In [3]:
def make_contour(im,cv2mode):

    img2gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(img2gray, (7,7), 0)

    ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,3))

    dilation = cv2.dilate(mask,kernel,iterations = 2)


    contours, hierarchy = cv2.findContours(dilation, mode=cv2mode, method=cv2.CHAIN_APPROX_SIMPLE) 
    #cv2mode = cv2.RETR_EXTERNAL , RETR_TREE
    
    return contours,hierarchy

def ocr_core(file):
    text = pytesseract.image_to_string(file,lang='tha+eng',config='-c preserve_interword_spaces=1')
    return text

In [4]:
im = cv2.imread('/Users/nichada/MyCode/Etda/pan-OCR/images/PO-150.png')

## - 5. - Crop New area by coordinate

- once we found the coordinate of the area of interest from the previous step 4.

In [5]:
#customise location

corr_ls= [(53, 300, 610, 100) #seller
        ,(53, 100, 610, 200) #address
        ,(900,100,300,200) #doc_no
                 ]

ocr_ls= []


for i in range(len(corr_ls[:])):
    x,y,w,h = corr_ls[i:i+1][0]
    
    # I use index like this because find_area func was set to accept list of tuples
    find_area(corr_ls[i:i+1],'/Users/nichada/MyCode/Etda/pan-OCR/images/PO-150.png')

    # crop image before ocr
    cropped = im[y :y +  h , x : x + w]

    # ocr image
    ocr_crop = ocr_core(cropped)

    # pre-processing the ocr string
    ocr_crop= ocr_crop.replace('\n',' ')
    
    ocr_ls.append(ocr_crop)

In [6]:
ocr_ls

['ผู้ขาย/ผู้รับจ้าง บริษัท ไทยไลฟ์สตรีมเอกซ์ จํากัด  1201/44 ซอยลาดพร้าว 94 (ปัญจมิตร) แขวงพลับพลา เขต วังทองหลาง กรุงเทพฯ 10310',
 'อาคารเดอะ ไนน์ ทาวเวอร์ แกรนด์ พระรามเก้า (อาคารบี) ชั้น 20-22 เลขที่ 33/4 ถนนพระราม 9 แขวงห้วยขวาง เขตห้วยขวาง กรุงเทพฯ 10310  โทรศัพท์       0-2123-1234      โทรสาร  www.etda.or.th เลขที่ประจําตัวผู้เสียภาษี (ของสพธอ.) 0994000789637',
 'AWwSsa  www.etda.or.th  ใบสิ่งซื้อ/จ้าง  เลขที่ 06401806 วันที่        22/09/2021  หน้า      1 จาก 1']

In [7]:
print(ocr_crop)

AWwSsa  www.etda.or.th  ใบสิ่งซื้อ/จ้าง  เลขที่ 06401806 วันที่        22/09/2021  หน้า      1 จาก 1


In [8]:
#regex
def regex_str(regex_,string_):
    result = re.search(regex_, string_)
    if result == None:
        return "n/a"
    return result[0]

In [9]:
seller = regex_str(r'(บริษัท|อาคาร|เลขที่|\d).* \b[0-9]{5}\b', ocr_ls[0])
seller

'บริษัท ไทยไลฟ์สตรีมเอกซ์ จํากัด  1201/44 ซอยลาดพร้าว 94 (ปัญจมิตร) แขวงพลับพลา เขต วังทองหลาง กรุงเทพฯ 10310'

In [10]:
address = regex_str(r'(บริษัท|อาคาร|เลขที่|\d).* \b[0-9]{5}\b', ocr_ls[1])
address

'อาคารเดอะ ไนน์ ทาวเวอร์ แกรนด์ พระรามเก้า (อาคารบี) ชั้น 20-22 เลขที่ 33/4 ถนนพระราม 9 แขวงห้วยขวาง เขตห้วยขวาง กรุงเทพฯ 10310'

In [11]:
doc_no = regex_str(r'(เลขที่|\d).* [0-9]{4,} ',ocr_ls[2])
doc_no

'เลขที่ 06401806 '

In [12]:
#save dict
dt_schema = {}
dt_schema = {'type' : 'string',
            'seller' : seller,
            'address' : address,
            'doc_no' : doc_no}

In [13]:
dt_schema

{'type': 'string',
 'seller': 'บริษัท ไทยไลฟ์สตรีมเอกซ์ จํากัด  1201/44 ซอยลาดพร้าว 94 (ปัญจมิตร) แขวงพลับพลา เขต วังทองหลาง กรุงเทพฯ 10310',
 'address': 'อาคารเดอะ ไนน์ ทาวเวอร์ แกรนด์ พระรามเก้า (อาคารบี) ชั้น 20-22 เลขที่ 33/4 ถนนพระราม 9 แขวงห้วยขวาง เขตห้วยขวาง กรุงเทพฯ 10310',
 'doc_no': 'เลขที่ 06401806 '}

In [14]:
# put in csv
file_dir = '/Users/nichada/MyCode/Etda/pan-OCR/images/ocr_schema'
if not os.path.exists(file_dir):
        os.mkdir(file_dir)
        
t = file_dir + '/P-150schm'
dt = dt_schema

with open(t+'.csv', "w") as csv_file:
    writer = csv.writer(csv_file)
    for c, values in dt.items():
        writer.writerow([c, values])

## (Extra) Save the cropped zone

- note : not finished

In [ ]:
def crop_ocrzone_n_save(im, corr_ls, foldername):
    cropfilename_ls = []
    for index,corr in enumerate(corr_ls):
        x,y,w,h = corr

        # crop image
        cropped = im[y :y +  h , x : x + w]

        # save crop path
        if not os.path.exists(f'images/{foldername}'):
            os.mkdir(f'images/{foldername}')

        s = f'images/{foldername}/crop_{index}.jpg'
        cropfilename_ls.append(s)
        
        cv2.imwrite(s , cropped)
        
        return cropfilename_ls

In [ ]:
seller_crop = crop_ocrzone_n_save(im,corr_ls, 'ocr_zone')

In [ ]:
print(seller_crop)

In [ ]:
r = '/Users/nichada/MyCode/Etda/pan-OCR/'+seller_crop[0]
seller_im = cv2.imread(r)


In [ ]:
seller_con, seller_hier = make_contour(seller_im, cv2.RETR_CCOMP)

In [ ]:
print(seller_hier[0])

In [ ]:
seller_im_copy = seller_im.copy()
#set contour here
contours= seller_con


for index,c in enumerate(seller_con[0][10:]):
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(seller_im_copy, (x, y), (x + w, y + h), (36,255,12), 2)
    cv2.imshow('crop', seller_im_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows() # destroys the window showing image
    cv2.waitKey(1) #for mac OS

In [ ]:
seller_im_copy = seller_im.copy()

x,y,w,h = 3, -1, -1, 1
cv2.rectangle(seller_im_copy, (x, y), (x + w, y + h), (36,255,12), 2)
cv2.imshow('crop', seller_im_copy)
cv2.waitKey(0)
cv2.destroyAllWindows() # destroys the window showing image
cv2.waitKey(1) #for mac OS